# HTML解析入门及准备URL生成连续技
![for humans](https://requests-html.kennethreitz.org/_static/requests-html-logo.png#thumbnail)

*  本周主要内容：HTML解析（parse HTML）及准备URL生成连续技
*  上周主要内容：HTML解析（parse HTML）及Xpath实践
*  20春_Web数据挖掘_week03
*  电子讲义设计者：廖汉腾, 许智超
<br/>
<br/>

-----
## 复习

复习：上周内容，实践

* HTML解析（parse HTML）: requests-html  丶
* Xpath实践
* m.liepin.com 取工作牛肉

-----
## 本周内容及学习目标

本周内容聚焦在

<mark> 如何从一页开始有系统的找更多页的内容 </mark>

为此，我们需要学习

1. 拆解带有参数的URL，并再从query取出参数
   a. URL拆解: 使用 urllib.parse 解析 出query
   b. query拆解:  取出参数 成python字典
2. 有基底URL，加上参数字典，请求新网页连续技

我们除了继续学习解决上一周已开始面对的以下挑战：
![Xpath Axis](http://krum.rz.uni-mannheim.de/inet-2005/images/xpath-axis.gif)

### 旧目标
1. 使用 requests-html 爬取并存取网页文字档，查找[requests-html 中文文档](https://cncert.github.io/requests-html-doc-cn/#/)
2. 熟悉 [xpath 语法](https://www.w3cschool.cn/xpath/xpath-syntax.html)丶[xpath 节点](https://www.w3cschool.cn/xpath/xpath-nodes.html)
3. 使用 [xpath cheatsheet](https://devhints.io/xpath)
  * 在 Chrome Inspector 使用
  * 在 requests-html (Python) 使用
4. 简易使用 [pd.DataFrame](https://www.pypandas.cn/doc/getting_started/dsintro.html#dataframe)

### 新目标
这一周，学生将实践
* 猎聘PC版 liepin.com 取工作URL参数的牛肉
* 如何生成一连串新URL以进一步爬取数据




In [14]:
%%html
<style>
/* 本电子讲义使用之CSS */
div.code_cell {
    background-color: #e5f1fe;
}
div.cell.selected {
    background-color: #effee2;
    font-size: 2rem;
    line-height: 2.4rem;
}
div.cell.selected .rendered_html table {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html pre code {
    background-color: #C4E4ff;   
    padding: 2px 25px;
}
.rendered_html pre {
    background-color: #99c9ff;
}
div.code_cell .CodeMirror {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html img, .rendered_html svg {
    max-width: 60%;
    height: auto;
    float: right;
}

.rendered_html img[src*="#full"], .rendered_html svg[src*="#full"] {
    max-width: 100%;
    height: auto;
    float: none;
}

.rendered_html img[src*="#thumbnail"], .rendered_html svg[src*="#thumbnail"] {
    max-width: 15%;
    height: auto;
}

/* Gradient transparent - color - transparent */
hr {
    border: 0;
    border-bottom: 1px dashed #ccc;
}
.emoticon{
    font-size: 5rem;
    line-height: 4.4rem;
    text-align: center;
    vertical-align: middle;
}
.bg-split_apply_comine {
    width: 500px;     
    height: 300px;
    background: url('02_split-apply-comine_500x300.png') -10px -10px;
    float: right;
}
.bg-comine {
    width: 175px;
    height: 150px;
    background: url('02_split-apply-comine_500x300.png') -280px -80px;
    float: right;
}
.bg-apply {
    width: 155px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -160px -30px;
    float: right;
}
.bg-split {
    width: 205px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -10px -30px;
    float: right;
}
.break {
                   page-break-after: right; 
                   width:700px;
                   clear:both;
}
</style>

In [15]:
# 基本模块
import pandas as pd
from requests_html import HTMLSession

# 0. 上周加分作业解答

In [16]:
# C-1   单一页面
url = "https://m.liepin.com/zhaopin/?keyword=PRD"
session = HTMLSession()
r = session.get( url )

# C-5
# 难: '公司URL', '时间', '经验'

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//div[@class="job-card-wrap"]//div[@class="job-card"]')

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        '经验':      './/ul/li[time]/text()'
    },
    'text_content': {
        '职称':    './/ul/li/a[contains(@class,"job-name")]/span[@class="name-text"]', 
        '薪水':    './/ul/li/a[contains(@class,"job-name")]/following-sibling::span', 
        '公司地点':'.//ul/li/time/following-sibling::a',
        '公司名称': './/ul/li/a[contains(@class,"company-name")]', 
        '时间':    './/ul/li/time', 
    },
    'href': {
        '链结':    './/ul/li/a[contains(@class,"job-name")]', 
        '公司URL': './/ul/li/a[contains(@class,"company-name")]', 
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
               if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

print ([len(v) for k,v in 数据字典.items()])  # 檢查

数据 = pd.DataFrame(数据字典)
数据.to_excel("20春_Web数据挖掘_week02_liepin.xlsx", sheet_name="搜查结果")
数据 

[60, 60, 60, 60, 60, 60, 60, 60]


,职称,薪水,公司地点,公司名称,时间,经验,链结,公司URL
0,旅游产品经理,12-20k·12薪,广州,前海爱讯科技(深圳)有限公司,16小时前,2年以上 统招本科,https://m.liepin.com/job/1926703515.shtml,https://m.liepin.com/company/8972310/
1,教育科技 软件产品经理,12-18k·12薪,广州,融捷投资控股集团,23小时前,3年以上 统招本科,https://m.liepin.com/job/1922705123.shtml,https://m.liepin.com/company/8025674/
2,产品经理,12-18k·12薪,广州-海珠区,广州大白互联网科技有限公司,22小时前,2年以上 本科及以上,https://m.liepin.com/job/1922402715.shtml,https://m.liepin.com/company/8695948/
3,实施经理,16-23k·12薪,广州-大沙,广东卓志供应链服务集团有限公司,2020-03-23,5年以上 统招本科,https://m.liepin.com/job/1924985573.shtml,https://m.liepin.com/company/9238204/
4,互联网产品经理,10-15k·12薪,广州-琶洲,广东车海洋环保科技有限公司,2020-03-20,3年以上 大专及以上,https://m.liepin.com/job/1917453193.shtml,https://m.liepin.com/company/9256869/
5,后台产品经理,10-20k·12薪,广州,广东南方新媒体股份有限公司,一个月前,3年以上 本科及以上,https://m.liepin.com/job/1925126353.shtml,https://m.liepin.com/company/7889168/
6,区块链产品经理,15-25k·12薪,广州-黄埔区,北京普瑞未来教育科技集团有限公司,一个月前,3年以上 大专及以上,https://m.liepin.com/job/1919835727.shtml,https://m.liepin.com/company/9989029/
7,高级产品经理,20-25k·13薪,广州,某软件开发企业,昨天,3年以上 大专及以上,https://m.liepin.com/a/18948933.shtml,
8,产品经理（电商系统）,25-40k·14薪,"广东,深圳,广州",知名跨境电商公司,昨天,3年以上 统招本科,https://m.liepin.com/a/18705133.shtml,
9,WMS产品经理,20-35k·14薪,广州,某知名跨境电商平台,一个月前,2年以上 学历不限,https://m.liepin.com/a/18963147.shtml,



-----

# 本周目标
* [猎聘PC版](https://www.liepin.com/zhaopin/)
* 上方导航有  公司行业 城市 薪资 的分页选单
* 请练习xpath抽出数据

## Xpath解析HTML

In [17]:
# A-1   单一页面
url = "https://www.liepin.com/zhaopin/?keyword=PRD"
session = HTMLSession()
r = session.get( url )

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//ul[@class="sojob-list"]/li')

# 预期是一个元素的列表？
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]'))
#print (主要元素[0].xpath('//div[contains(@class,"job-info")]/h3/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="text-warning"]'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/time/@title'))
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a'))

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
        '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
        '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
        '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
        '职称':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司地点': '//div[contains(@class,"job-info")]/p/a',
        '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    },
    'text_content': {
    },
    'href': {
        '链结':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
               if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

[len(v) for k,v in 数据字典.items()]

数据 = pd.DataFrame(数据字典)
数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
数据 

,edu,经验,薪水,时间,职称,公司地点,公司名称,链结,公司URL
0,统招本科,5年以上,15-25k·12薪,2020年03月31日,财务管理部长,重庆-永川区,山东宏鑫源钢板有限公司,https://www.liepin.com/job/1927090305.shtml,https://www.liepin.com/company/8030311/
1,本科及以上,10年以上,12-20k·12薪,2020年03月31日,集团工程经理,南宁-青秀区,华劲集团股份有限公司,https://www.liepin.com/job/1927027679.shtml,https://www.liepin.com/company/7938232/
2,本科及以上,3年以上,面议,2020年03月31日,嵌入式硬件工程师,南京-江宁区,南高齿集团,https://www.liepin.com/job/1927014087.shtml,https://www.liepin.com/company/8037821/
3,本科及以上,3年以上,面议,2020年03月31日,寻源采购工程师-国际采购,南京-江宁区,南高齿集团,https://www.liepin.com/job/1927006131.shtml,https://www.liepin.com/company/8037821/
4,统招本科,8年以上,8-10k·14薪,2020年03月31日,财务分析主管,镇江-丹阳,大亚科技集团,https://www.liepin.com/job/1926767325.shtml,https://www.liepin.com/company/7916453/
5,本科及以上,8年以上,面议,2020年03月31日,应用架构师,南宁-青秀区,华劲集团股份有限公司,https://www.liepin.com/job/1926721779.shtml,https://www.liepin.com/company/7938232/
6,本科及以上,5年以上,12-15k·12薪,2020年03月31日,国企行政副总,威海-环翠区,烟台市方程式软件有限公司,https://www.liepin.com/job/1926176983.shtml,https://www.liepin.com/company/9987923/
7,本科及以上,5年以上,13-17k·12薪,2020年03月31日,政府园区招商副总,威海-环翠区,烟台市方程式软件有限公司,https://www.liepin.com/job/1926176943.shtml,https://www.liepin.com/company/9987923/
8,本科及以上,7年以上,面议,2020年03月31日,(Sr.) Sales Account Manager,上海-浦东新区,MATLAB,https://www.liepin.com/job/1925634897.shtml,https://www.liepin.com/company/8185522/
9,本科及以上,5年以上,10-15k·12薪,2020年03月31日,绩效经理,东莞,正业科技广东,https://www.liepin.com/job/1925579885.shtml,https://www.liepin.com/company/470396/


## 使用urllib3 解析 url 
上面的url应该触动不同的页面查询，但能不能轻松无误的拆分url并进行比较？

### urllib模块功能介绍
* urlparse 
返回的6个部分，分别是：scheme(机制)丶netloc(网络位置)丶path(路径)丶params(路径段参数)丶query(查询)丶fragment(片段)。
* parse_qs
返回query(查询)多个部分

In [6]:
# B-1 使用 urllib.parse 解析
from urllib.parse import urlparse, parse_qs
[ urlparse(x) for x in 公司数据选择器链结.values()]

[ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=1e6dae188c31324c&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=155&ckid=1e6dae188c31324c&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=7de6fbaf0e194e58f87331cdb1a9a10c&d_curPage=0&d_pageSize=40&d_headId=7de6fbaf0e194e58f87331cdb1a9a10c', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=1e6dae188c31324c&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=182&ckid=1e6dae188c31324c&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=7de6fbaf0e194e58f87331cdb1a9a10c&d_curPage=0&d_pageSize=40&d_headId=7de6fbaf0e194e58f87331cdb1a9a10c', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=1e6dae188c31324c&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=186&ckid=1e6dae188c31324c&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom

In [18]:
# A-2 扩张 公司 ?  

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath('//div[@data-selector="search-conditions"]')
# 预期是一个元素的列表？
print (主要元素)
print (主要元素[0])
print (主要元素[0].xpath('//dt[@class="search-title"]'))

list_search_title = 主要元素[0].xpath('//dt[@class="search-title"]')
for x in list_search_title:
    print (x.text)
    
list_search_dd = 主要元素[0].xpath('//dt[@class="search-title"]/following-sibling::dd')
for x in list_search_dd:
    print (x)  
    

公司数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a/@href')
               
公司数据选择器链结

# 但我们需要知道这些选择器链结, 对映到什麽数据
公司数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a')
公司数据选择器链结

#[ x.xpath("a/@href")[0] for x in 公司数据选择器链结]
#[ x.xpath("a/text()")[0] for x in 公司数据选择器链结]
公司数据选择器链结 = { x.xpath("a/text()")[0]:x.xpath("a/@href")[0] for x in 公司数据选择器链结}
公司数据选择器链结

[<Element 'div' class=('search-conditions',) data-selector='search-conditions'>]
<Element 'div' class=('search-conditions',) data-selector='search-conditions'>
[<Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>]
公司：
行业：
城市：
薪资：
更多：
<Element 'dd' class=('comp-list',)>
<Element 'dd' class=('short-dd', 'select-industry') data-param='industries'>
<Element 'dd' data-param='city'>
<Element 'dd' data-param='salary'>
<Element 'dd' class=('dropdown', 'dropdown-time')>
<Element 'dd' class=('dropdown', 'dropdown-jobkind')>
<Element 'dd' class=('dropdown', 'dropdown-compscale')>
<Element 'dd' class=('dropdown', 'dropdown-compkind')>


{'中国500强': '/zhaopin/?init=-1&headckid=609e717fd0beeed7&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=155&ckid=609e717fd0beeed7&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=07e09d3bd0b0a5edb9a91ba3508a8f3e&d_curPage=0&d_pageSize=40&d_headId=07e09d3bd0b0a5edb9a91ba3508a8f3e',
 '2018互联网300强': '/zhaopin/?init=-1&headckid=609e717fd0beeed7&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=182&ckid=609e717fd0beeed7&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=07e09d3bd0b0a5edb9a91ba3508a8f3e&d_curPage=0&d_pageSize=40&d_headId=07e09d3bd0b0a5edb9a91ba3508a8f3e',
 '制造业500强': '/zhaopin/?init=-1&headckid=609e717fd0beeed7&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=186&ckid=609e717fd0beeed7&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=07e09d3bd0b0a5edb9a91ba3508a8f3e&d_curPage=0&d_pageSize=40&d_headId=07e09d3bd0b0a5edb9a91ba3508a8f3e',
 'AI创新成长50强 ': '/zhaopin/?init=-1&headckid=

In [21]:
# B-2 使用 pd.DataFrame进行 unuinque()相异值计量比对 
import pandas as pd
df = pd.DataFrame([ urlparse(x) for x in 公司数据选择器链结.values()])
df.info()
print(df.nunique())
df.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   scheme    6 non-null      object
 1   netloc    6 non-null      object
 2   path      6 non-null      object
 3   params    6 non-null      object
 4   query     6 non-null      object
 5   fragment  6 non-null      object
dtypes: object(6)
memory usage: 416.0+ bytes
scheme      1
netloc      1
path        1
params      1
query       6
fragment    1
dtype: int64


,scheme,netloc,path,params,query,fragment
0,,,/zhaopin/,,init=-1&headckid=609e717fd0beeed7&flushckid=1&...,


In [22]:
# B-3 针对query 再解析之 
#df_qs = pd.DataFrame([ parse_qs(x) for x in df['query'] ])
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
print (df_qs.nunique())
df_qs.head()
df_qs[['keyword','compTag']]

init             1
headckid         1
flushckid        1
fromSearchBtn    1
keyword          1
compTag          6
ckid             1
siTag            1
d_sfrom          1
d_ckId           1
d_curPage        1
d_pageSize       1
d_headId         1
dtype: int64


,keyword,compTag
0,PRD,155
1,PRD,182
2,PRD,186
3,PRD,189
4,PRD,130
5,PRD,156


### 小结
* comTag 是不同的公司选择器, 数值不样, 对映到不同类型的公司
* keyword 是搜查关键字

In [9]:
# B-4 建构 参数模板 及 字典_compTag
def parse_url_qs_for_compTag (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

# parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])['compTag']
参数模板 = parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])
print(参数模板)
# [ parse_url_qs_for_compTag(x)['compTag'] for x in 公司数据选择器链结.values()]
[ parse_url_qs_for_compTag(x)['compTag'][0] for x in 公司数据选择器链结.values()]

字典_compTag = { k:parse_url_qs_for_compTag(v)['compTag'][0] for k,v in 公司数据选择器链结.items()}
print (字典_compTag)


{'init': ['-1'], 'headckid': ['1e6dae188c31324c'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['PRD'], 'compTag': ['155'], 'ckid': ['1e6dae188c31324c'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['7de6fbaf0e194e58f87331cdb1a9a10c'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['7de6fbaf0e194e58f87331cdb1a9a10c']}
{'中国500强': '155', '2018互联网300强': '182', '制造业500强': '186', 'AI创新成长50强 ': '189', '独角兽': '130', '上市公司': '156'}


In [10]:
# B-5 建构 参数模板  
def 参数模板生成(compTag , keyword ):
    参数 = 参数模板.copy()
    参数['compTag'] = compTag
    参数['keyword'] = keyword
    return (参数)

参数_compTag_用户体验 = { k:参数模板生成(compTag = [v], keyword = ['用户体验']) for k,v in 字典_compTag.items()}
print(参数_compTag_用户体验)

{'中国500强': {'init': ['-1'], 'headckid': ['1e6dae188c31324c'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['155'], 'ckid': ['1e6dae188c31324c'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['7de6fbaf0e194e58f87331cdb1a9a10c'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['7de6fbaf0e194e58f87331cdb1a9a10c']}, '2018互联网300强': {'init': ['-1'], 'headckid': ['1e6dae188c31324c'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['182'], 'ckid': ['1e6dae188c31324c'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['7de6fbaf0e194e58f87331cdb1a9a10c'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['7de6fbaf0e194e58f87331cdb1a9a10c']}, '制造业500强': {'init': ['-1'], 'headckid': ['1e6dae188c31324c'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['186'], 'ckid': ['1e6dae188c31324c'], '

## requests 生成

In [23]:
# C-1   多个页面准备测试1 中国500强
url = "https://www.liepin.com/zhaopin/"
session = HTMLSession()
payload = 参数_compTag_用户体验['中国500强']
r = session.get( url, params = payload)
r.url

'https://www.liepin.com/zhaopin/?init=-1&headckid=1e6dae188c31324c&flushckid=1&fromSearchBtn=2&keyword=%E7%94%A8%E6%88%B7%E4%BD%93%E9%AA%8C&compTag=155&ckid=1e6dae188c31324c&siTag=1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=7de6fbaf0e194e58f87331cdb1a9a10c&d_curPage=0&d_pageSize=40&d_headId=7de6fbaf0e194e58f87331cdb1a9a10c'

In [24]:
# C-2  简化 A-1   单一页面爬+解析
session = HTMLSession()

def requests_liepin( url, params):
    r = session.get( url , params = payload)

    # 先取特定元素, 精准打击其子后辈
    主要元素 = r.html.xpath( '//ul[@class="sojob-list"]/li')

    # 作为xpath字典，键为我要抓的牛肉名称，值为xpath
    dict_xpaths={ 
        'text': {
            'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
            '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
            '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
            '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
            '职称':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司地点': '//div[contains(@class,"job-info")]/p/a',
            '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        },
        'text_content': {
        },
        'href': {
            '链结':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        }
    }

    def get_e_text_content(_xpath_):
        # 高级列表推导
        暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
        return(暂存结果)

    def get_e_text(_xpath_):
        # 高级列表推导
        暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
        return(暂存结果)

    def get_e_href(_xpath_):
        # 高级列表推导
        暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                   if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                   else "" for e in 主要元素]
        return(暂存结果)

    # 只对主要元素下进行.xpath取值
    数据字典 = dict()

    数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
    数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
    数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

    数据 = pd.DataFrame(数据字典)
    #数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
    return (数据)




In [25]:
# C-3   多个页面
url = "https://www.liepin.com/zhaopin/"

list_df = list()
for k,v in 参数_compTag_用户体验.items():
    payload = v
    df = requests_liepin( url, params = payload)
    df = df.assign (热门公司类型 = k)    
    list_df.append(df)

df_all = pd.concat(list_df)
df_all

,edu,经验,薪水,时间,职称,公司地点,公司名称,链结,公司URL,热门公司类型
0,本科及以上,3年以上,面议,2020年03月30日,大客户销售经理-北京-网易严选,北京-五道口,网易集团,https://www.liepin.com/job/1926756751.shtml,https://www.liepin.com/company/5964833/,中国500强
1,统招本科,3年以上,面议,2020年03月30日,阿里云智能事业群-数据技术专家(金融行业)-北京/杭州,杭州,阿里巴巴,https://www.liepin.com/job/1927063431.shtml,https://www.liepin.com/company/1072424/,中国500强
2,本科及以上,3年以上,面议,2020年03月27日,钉钉(Dingtalk)-搜索中心-Java开发技术专家,杭州,阿里巴巴,https://www.liepin.com/job/1926996383.shtml,https://www.liepin.com/company/1072424/,中国500强
3,大专及以上,2年以上,6-8k·13薪,2020年03月25日,员工关系专员,廊坊-广阳区,中国国际技术智力合作有限公司,https://www.liepin.com/job/1926938099.shtml,https://www.liepin.com/company/1233751/,中国500强
4,本科及以上,3年以上,25-50k·12薪,2020年03月24日,钉钉(DingTalk)-安全运营专家-安全产品及中心,杭州,阿里巴巴,https://www.liepin.com/job/1926923363.shtml,https://www.liepin.com/company/1072424/,中国500强
...,...,...,...,...,...,...,...,...,...,...
35,本科及以上,6年以上,15-20k·13薪,2020年03月26日,法务经理/主任,深圳,中国南玻集团股份有限公司,https://www.liepin.com/job/1926955487.shtml,https://www.liepin.com/company/9091167/,上市公司
36,统招本科,10年以上,面议,2020年03月26日,CHO/HRD,上海,银科控股,https://www.liepin.com/job/1915800458.shtml,https://www.liepin.com/company/8582797/,上市公司
37,本科及以上,3年以上,20-30k·12薪,2020年03月25日,SAP 运维顾问,北京,科兴,https://www.liepin.com/job/1926949105.shtml,https://www.liepin.com/company/8593199/,上市公司
38,本科及以上,1年以上,10-20k·12薪,2020年03月25日,新闻短视频运营 (MJ000067),北京,凤凰新媒体,https://www.liepin.com/job/1925965933.shtml,https://www.liepin.com/company/8139695/,上市公司


In [26]:
# C-4   输出
df_all.to_excel("20春_Web数据挖掘_week03_liepin_各热门公司类型.xlsx", sheet_name="搜查结果")

In [27]:
# C-5 Pandas  基本能力

print (df_all.nunique())
df_all[['edu']].drop_duplicates()

df_all.groupby(['公司名称','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

edu         7
经验         10
薪水         77
时间         31
职称        186
公司地点       72
公司名称       56
链结        197
公司URL      57
热门公司类型      6
dtype: int64


职称
公司名称            edu      
华为              统招本科   30
                本科及以上  18
网易集团            本科及以上  17
科大讯飞            本科及以上  14
盛虹控股集团有限公司      统招本科   10
...                    ..
深圳市优必选科技股份有限公司  本科及以上   1
天津深之蓝海洋设备科技有限公司 本科及以上   1
深圳视见医疗科技有限公司    大专及以上   1
天津深之蓝海洋设备科技有限公司 博士      1
龙信集团            统招本科    1

[83 rows x 1 columns]

In [28]:
df_all.groupby(['公司名称','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

职称
公司名称            edu      
华为              统招本科   30
                本科及以上  18
网易集团            本科及以上  17
科大讯飞            本科及以上  14
盛虹控股集团有限公司      统招本科   10
...                    ..
深圳市优必选科技股份有限公司  本科及以上   1
天津深之蓝海洋设备科技有限公司 本科及以上   1
深圳视见医疗科技有限公司    大专及以上   1
天津深之蓝海洋设备科技有限公司 博士      1
龙信集团            统招本科    1

[83 rows x 1 columns]

# 本周练习

一样反向工程解析:

## 上方界面的params参数
* 公司：v
* 行业：?
* 城市：?
* 薪资：?
## 下方界面的params参数
* 跳转到 N 页确定 ?
## 换  
* keyword
